# 1. 分析输入数据的基本情况
- 数据类型：定类/定序/等差/等比，数值/非数值
- 数值的取值范围和分布
- 缺失值

In [1]:
train_csv = './data/train.csv'
test_csv = './data/test.csv'

In [2]:
import pandas as pd

df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [3]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [5]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
mod_df = df.mode().iloc[0]

In [7]:
# 查看存在nan值的列
for column in df.columns:
    c = len(df[df[column].isna()])
    print(f'{column}:{c}')

PassengerId:0
Survived:0
Pclass:0
Name:0
Sex:0
Age:177
SibSp:0
Parch:0
Ticket:0
Fare:0
Cabin:687
Embarked:2


In [8]:
df['Survived'].sum()

342

# 2.获取基线
- 根据类别占比计算基线1
- 简单特征工程和模型计算基线2

## （1）baseline1
- 使用数量多的类别作为预测结果

In [9]:
baseline1 = (len(df) - df['Survived'].sum()) / len(df)
print(baseline1)

0.6161616161616161


## （2）baseline2
- 简单的特征工程+模型获取基线

### 数据预处理
- 填充缺失值

In [10]:
df = df.fillna(mod_df)

### 特征工程
- Pclass: 转为one-hot
- Name: 字符串类型，先不使用该特征
- sex: 0-1
- Age: 浮点数，使用众数填充na
- SibSp: 浮点数
- Parch: 浮点数
- Ticket: 先不用
- Fare: 浮点数
- Cabin: 字符串类型，缺失值多，先不用
- Embarked: 转为one-hot，使用众数填充na

In [12]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

onehot_columns = ['Sex', 'Pclass', 'Embarked']
value_columns = ['Age', 'SibSp', 'Parch', 'Fare']

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df[onehot_columns])

enc_names = enc.get_feature_names_out(onehot_columns).tolist()
fea_names = enc_names + value_columns

def baseline_fea(df):
    ori_onehot_fea = df[onehot_columns].values
    value_fea = df[value_columns].values
    
    onehot_fea = enc.transform(ori_onehot_fea).toarray()
    
    fea = np.concatenate([onehot_fea, value_fea], axis=1)
    
    return fea
    
    
fea = baseline_fea(df)
lab = df.Survived.values

/Users/zhushuguang/Software/miniconda3/envs/python38/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


### 模型训练
- 选择xgb二分类

In [18]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import random


index = list(range(len(fea)))
random.shuffle(index)
train_ratio = 0.8
train_fea = fea[index[:int(len(fea) * 0.8)]]
val_fea = fea[index[int(len(fea) * 0.8):]]
train_lab = lab[index[:int(len(fea) * 0.8)]]
val_lab = lab[index[int(len(fea) * 0.8):]]

train_data = xgb.DMatrix(train_fea, label=train_lab)
val_data = xgb.DMatrix(val_fea, label=val_lab)

param = {'max_depth': 4, 'eta': 0.3, 'objective': 'binary:logistic'}
num_round = 100


In [19]:
bst = xgb.train(param, train_data, num_round)

[18:15:00] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [22]:
val_prob = bst.predict(val_data)

In [29]:
thres = 0.5
val_predict = np.array([1 if p >= thres else 0 for p in val_prob])
acc = (val_predict == val_lab).sum() / len(val_lab)
print(acc)

0.7932960893854749


In [37]:
# 测试集结果
import pandas as pd

test_df = test_df.fillna(mod_df)

test_fea = baseline_fea(test_df)
test_fea = xgb.DMatrix(test_fea)
test_prob = bst.predict(test_fea)

test_predict = np.array([1 if p >= thres else 0 for p in test_prob])
PassengerId = test_df.PassengerId.values.tolist()

submit_df = pd.DataFrame({'PassengerId': PassengerId, 'Survived': test_predict})
submit_df.to_csv('baseline1.csv', header=True, index=False)

/Users/zhushuguang/Software/miniconda3/envs/python38/lib/python3.8/site-packages/sklearn/base.py:441: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
